# Model Selection Module
- train data, test data 분리하지 않고 머신러닝 수행

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
# x,y 데이터
dt_clf.fit(iris.data , iris.target)

DecisionTreeClassifier()

In [3]:
pred = dt_clf.predict(iris.data)
# y실제값, y햇
accuracy_score(iris.target, pred)

1.0

- 학습 데이터와 테스트 데이터(30%) 분리

In [6]:
from sklearn.model_selection import train_test_split

In [11]:
# X_train, X_test, y_train, y_test 순서 중요
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size = 0.3, random_state = 121 
    # random_state: 랜덤으로 셔플할 때 사용되는 시드값
)

In [12]:
dt_clf = DecisionTreeClassifier()
# x,y 데이터
dt_clf.fit(X_train , y_train)

DecisionTreeClassifier()

In [15]:
pred = dt_clf.predict(X_test)
# y실제값, y햇
accuracy_score(y_test, pred)

0.9555555555555556

# 교차 검증
- 학습 데이터 셋, 테스트 데이터 셋 이렇게 둘로만 나누지 않고 검증 데이터 셋도 추가

In [16]:
from sklearn.model_selection import KFold
import numpy as np
kfold = KFold(n_splits=5)
cv_accuracy = [] # 교차검증(Cross Validation) accuracy
features, label = iris.data, iris.target
dt_clf = DecisionTreeClassifier(random_state=156)
print('붓꽃 데이터 세트 크기:', features.shape[0])

for train_index, test_index in kfold.split(features):
 # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
 X_train, X_test = features[train_index], features[test_index]
 y_train, y_test = label[train_index], label[test_index]
 #학습 및 예측
 dt_clf.fit(X_train , y_train)
 pred = dt_clf.predict(X_test)
 n_iter += 1
 # 반복 시 마다 정확도 측정
 accuracy = np.round(accuracy_score(y_test, pred), 4)
 train_size = X_train.shape[0]
 test_size = X_test.shape[0]
 print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
 .format(n_iter, accuracy, train_size, test_size))
 print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
 cv_accuracy.append(accuracy)

print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

붓꽃 데이터 세트 크기: 150

#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


- setosa, versicolor, virginica 데이터가 50개씩 들어가 있는데 k-fold를 앞에서 30개 인덱스씩 뽑아옴<br>
세토사는 확연히 차이가 나기 때문에 1번세트처럼 세토사만 하면 1이 나옴<br>
버지칼라랑 버지니카는 데이터가 구분이 잘 안 되었기 때문에 둘이 섞이면 잘 안나옴


즉, y값이 불균형한 분포일 때는 kFold가 잘 안먹힌다.
이걸 해결하기 위해 나온 Stratified K-Fold

In [21]:
import pandas as pd
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [22]:
# 3개로 나눔. 일반 KFold
kfold = KFold(n_splits=3)
n_iter = 0
for train_index, test_index in kfold.split(iris_df):
 n_iter += 1
 label_train= iris_df['label'].iloc[train_index]
 label_test= iris_df['label'].iloc[test_index]
 print('## 교차 검증: {0}'.format(n_iter))
 print('학습 레이블 데이터 분포:\n', label_train.value_counts())
 print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [23]:
# 3개로 나눔. Stratified K Fold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3) # 요기만 다름
n_iter = 0
for train_index, test_index in skf.split(iris_df, iris_df['label']):
 n_iter += 1
 label_train= iris_df['label'].iloc[train_index]
 label_test= iris_df['label'].iloc[test_index]
 print('## 교차 검증: {0}'.format(n_iter))
 print('학습 레이블 데이터 분포:\n', label_train.value_counts())
 print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64


Stratified KFold 교차 검증

In [24]:
skfold = StratifiedKFold(n_splits=3)
cv_accuracy = []
features, label = iris.data, iris.target
n_iter = 0

dt_clf = DecisionTreeClassifier(random_state=156)

In [25]:
for train_index, test_index in skfold.split(features, label):
 # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
 X_train, X_test = features[train_index], features[test_index]
 y_train, y_test = label[train_index], label[test_index]
 #학습 및 예측
 dt_clf.fit(X_train , y_train)
 pred = dt_clf.predict(X_test)
 n_iter += 1
 # 반복 시 마다 정확도 측정
 accuracy = np.round(accuracy_score(y_test, pred), 4)
 train_size = X_train.shape[0]
 test_size = X_test.shape[0]
 print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
 .format(n_iter, accuracy, train_size, test_size))
 print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
 cv_accuracy.append(accuracy)


#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]


In [26]:
print('## 교차 검증:', np.round(cv_accuracy,4))
print('평균 검증 정확도:', np.mean(cv_accuracy))

## 교차 검증: [0.98 0.94 0.98]
평균 검증 정확도: 0.9666666666666667


- 편리한 교차 검증 세트

In [27]:
from sklearn.model_selection import cross_val_score

In [32]:
scores = cross_val_score(dt_clf, iris.data, iris.target, scoring='accuracy', cv=3) 
# dt_clf, x값, y값, scoring(=평가) = 'acuracy', 평가를 accuracy로 하겠다. cv = 3)
print('## 교차 검증:', np.round(scores,4))
print('평균 검증 정확도:', np.round(np.mean(scores),4))


## 교차 검증: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


### GridSearchCV - 교차 검증, 최적 하이퍼 파라미터 튜닝을 한 번에

In [37]:
parameters = {
    'max_depth': [2,3,4],
    'min_samples_split': [2,3]
}

In [46]:
from sklearn.model_selection import GridSearchCV

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size = 0.3, random_state = 121 
)

dtree = DecisionTreeClassifier()

In [47]:
grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv= 3,scoring = 'accuracy', refit = True) 
# 모델명, 하이퍼 파라미터, cv, scoring, refit

In [49]:
grid_dtree.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4], 'min_samples_split': [2, 3]},
             scoring='accuracy')

In [50]:
grid_dtree.cv_results_
# 잘 안보이니까 데이터프레임으로 만들어보자

{'mean_fit_time': array([0.00101876, 0.00083598, 0.00117413, 0.00099778, 0.00066582,
        0.00105476]),
 'std_fit_time': array([1.80485646e-05, 2.28830035e-04, 2.05849956e-04, 8.21085226e-04,
        4.70839947e-04, 8.01167810e-04]),
 'mean_score_time': array([0.0004189 , 0.        , 0.00053414, 0.00068331, 0.0003256 ,
        0.00084607]),
 'std_score_time': array([0.00059242, 0.        , 0.00041138, 0.00048318, 0.00046047,
        0.00062214]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_samples_split': 3},
  {'max_depth': 3, 'min_samples_split': 2},
  {'max_depth': 3, 'min_samples_split': 3},
  {'ma

In [51]:
score_df = pd.DataFrame(grid_dtree.cv_results_)
score_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 2, 'min_samples_split': 2}",0.933333,4,0.942857,0.914286,0.942857
1,"{'max_depth': 2, 'min_samples_split': 3}",0.933333,4,0.942857,0.914286,0.942857
2,"{'max_depth': 3, 'min_samples_split': 2}",0.942857,1,0.971429,0.914286,0.942857
3,"{'max_depth': 3, 'min_samples_split': 3}",0.942857,1,0.971429,0.914286,0.942857
4,"{'max_depth': 4, 'min_samples_split': 2}",0.942857,1,0.971429,0.914286,0.942857
5,"{'max_depth': 4, 'min_samples_split': 3}",0.933333,4,0.942857,0.914286,0.942857


In [52]:
grid_dtree.best_params_
# 하이퍼 파라미터를 이렇게 줬을 때 결과가 제일 좋더라

{'max_depth': 3, 'min_samples_split': 2}

In [53]:
grid_dtree.best_score_
# 그때의 score는?

0.9428571428571427